# Data Tidying and Manipulation in Python
## by David DeTomaso, Diya Das, and Andrey Indukaev

### The goal
Data tidying is a necessary first step for data analysis - it's the process of taking your messily formatted data (missing values, unwieldy coding/organization, etc.) and literally tidying it up so it can be easily used for downstream analyses. To quote Hadley Wickham, "Tidy datasets are easy to manipulate, model and visualise, and have a specific structure:
each variable is a column, each observation is a row, and each type of observational unit
is a table."

These data are actually pretty tidy, so we're going to be focusing on cleaning and manipulation, but these manipulations will give you some idea of how to tidy untidy data.

### The datasets
We are going to be using the data from the R package [`nycflights13`](https://cran.r-project.org/web/packages/nycflights13/nycflights13.pdf). There are five datasets corresponding to flights departing NYC in 2013. We will load directly into R from the library, but the repository also includes CSV files we created for the purposes of the Python demo and can also be used to load the data into our R session.



## Python requirements

For this tutorial we'll be using the following packages in Python
- pandas (depends on numpy)
- seaborn (depends on matplotlib)

You can install these with either `pip` or `conda`

### pandas

Pandas is an extremely useful package for data-manipulation in python.  It allows for a few things:

- Mixed types in a data matrix
- Non-numeric row/column indexes
- Database-like join/merge/group-by operations
- Data import/export to a variety of formats [(text, Excel, JSON)](http://pandas.pydata.org/pandas-docs/stable/api.html#input-output)

The core pandas object is a 'dataframe' - modeled after DataFrames in R

In [ ]:
from __future__ import print_function # For the python2 people
import pandas as pd # This is typically how pandas is loaded

## Reading data from a file
Let's read data from a file

There are five tables we'll be using as part of the NYCFlights13 dataset

To view them, first extract the archive that comes with this repo

```bash
unzip nycflights13.zip
```

Now, to read them in as dataframes, we'll use the **[read_table](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_table.html)** function from pandas

This is a general purpose function for reading tabular data in a text file format.  If you follow the link, you can see that there are many configurable options.  We'll just use the defaults (assumes tab-delimited)

In [ ]:
airlines = pd.read_table("airlines.txt")
airports = pd.read_table("airports.txt")
flights = pd.read_table("flights.txt")
planes = pd.read_table("planes.txt")
weather = pd.read_table("weather.txt")

## Inspecting a dataframe // What's in the `flights` dataset?
Let's run through an example using the `flights` dataset. This dataset includes...well what does it include? You could read the documentation, but let's take a look first.

### Anatomy of a pandas DataFrame

There are a couple of concepts that are important to understand when working with dataframes
- DataFrame class
- Series
- Index / Columns

To understand these, lets look at the 'planes' dataframe

In [ ]:
print(type(planes)) # Yup, it's a DataFrame

In [ ]:
# What does it look like?
planes # Jupyter Notebooks do some nifty formatting here

In [ ]:
# How big is it?
print(planes.shape) # Works like numpy

In [ ]:
print(planes.columns) # What are the column labels?

In [ ]:
print(planes.index) # What are the row labels?

In [ ]:
# Let's grab a column
planes['manufacturer']

In [ ]:
# Inspecting this column further
manufacturer = planes['manufacturer']
print(type(manufacturer)) # It's a Series

### Series
one-dimensional and only have one set of labels (called index)

### DataFrames 
two-dimensional and have row-labels (called index) and column-labels (called columns)
DataFrames are made up of Series (each column is a series)

In [ ]:
# Indexing into Series
print("Indexing into Series: ", manufacturer[3])

# Indexing into DataFrame
print("Indexing into DataFrame: ", planes.loc[3, 'manufacturer'])

### DataFrame Indexing

We already showed that you can grab a column using
```
dataframe[column_name]
```

To grab a row, use:
```
dataframe.loc[row_name]
```

And to grab a specific element use:
```
dataframe.loc[row_name, column_name]
```

In [ ]:
third_row = planes.loc[3] # get the third row
third_row

In [ ]:
print(type(third_row))

### Dataframe index

So far the row-index has been numeric (just 0 through ~3300).  However, we might want to use labels here too.

To do this, we can select a column to be the dataframe's index
**Only do this if the column contains unique data**

In [ ]:
planes = planes.set_index('tailnum')

# OR

planes = pd.read_table('planes.txt', index_col=0) #Set the first column as the index

planes.loc['N10156']

### But now how do I get the 3rd row?

Here's where **iloc** comes into play.

Works like **loc** but uses integers

In [ ]:
print(planes.iloc[3]) # Get the third row
print(planes.iloc[:, 3]) # Get the third column

## Exploring our dataset - let's look at the 'flights' table

In [ ]:
print('What are the first 5 rows?')
flights.head()

In [ ]:
print('What are the last 5 rows?')
flights.tail()

In [ ]:
print('Sample random rows')
flights.sample(3, axis=0) # Axis 0 represents the rows, axis 1, the columns 

## Identifying and removing NAs in a dataset
We noticed some NAs above (hopefully). How do you find them and remove observations for which there are NAs? 

In [ ]:
print('What are the dimensions of the flights dataframe?\n')
print(flights.shape)

print('Are there any NAs in the flights dataframe?\n')
print(flights.isnull().any())

In [ ]:
print('Selecting for flights where there is complete data, what are the dimensions?\n')
print("Original Matrix Shape:", flights.shape)
null_rows = flights.isnull().any(axis=1) # Rows where any value is null
flights_complete = flights.loc[~null_rows]
print("Complete-rows shape:", flights_complete.shape)

### Aside: Why does this work with loc?  

Earlier I showed .loc operating on row/column labels.

Well, it can also operate on boolean (true/false) lists (or numpy arrays, or **pandas Series**)

Above, what is null_rows?

In [ ]:
print(type(null_rows))
null_rows

The great thing about Pandas is that if you pass in a Series, the order of the elements in it doesn't matter anymore.  It uses the index to align the Series to the row/column index of the dataframe.

This is very useful when creating a boolean index from one dataframe to be used to select rows in another!

Alternately, with removing NA values there is a [dropna](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html) function that can be used.

Now...back to flights!

In [ ]:
print('How might I obtain a summary of the original dataset?')
flights.describe() # Similar to R's 'summary'
# use include='all' to include the non-numberic columns too

## Performing a function along an axis

Pandas allows easy application of descriptive function along an axis.

**any** which we used earlier, is an example of that.  If the data is boolean, any collapses a series of boolean values into True if *any* of the values are true (otherwise, False)

Can also use min, max, mean, var, std, count

In [ ]:
# An exmaple
flights['air_time'].mean() # Returns a single value

In [ ]:
subset = flights[['air_time', 'dep_delay', 'arr_delay']]
subset.mean(axis=0) # Axis 0: collapse all rows, result has Index = to original Columns

If you want to apply an arbitrary function along an axis, look into the [apply function](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html)

## Performing column-wise operations while grouping by other columns // Departure delay by airport of origin
Sometimes you may want to perform some aggregate function on data by category, which is encoded in another column. Here we calculate the statistics for departure delay, grouping by origin of the flight - remember this is the greater NYC area, so there are only three origins!

In [ ]:
result = flights_complete.groupby('origin')['dep_delay'].mean()
result

In [ ]:
# What is this object?
print(type(result))

Other descriptive functions work here, like 'std', 'count', 'min', 'max'

Also: describe

In [ ]:
flights_complete.groupby('origin')['dep_delay'].describe()

## Merging tables 'vertically' // Subsetting and re-combining flights from different airlines
You will likely need to combine datasets at some point. R provides quite a few tools to do that, and as you've seen, it's possible to do something many different ways.

Here, we present a simple case of 'vertical' merging, using `rbind`. Let's create a data frame with information on flights by United Airlines and American Airlines only, by creating two data frames via subsetting data about each airline one by one and then merging. 

The main requirement is that the columns must have the same names (may be in different order).

In [ ]:
message('Subsetting the dataset to have 2 dataframes')
flightsUA <- flights[flights$carrier == 'UA',]
flightsAA <- flights[flights$carrier == 'AA',]
message('Checking the number of rows in two dataframes')
print(nrow(flightsUA) + nrow(flightsAA))
message('Combining two dataframes than checking the number of rows in the resulting data frame')
flightsUAandAA <- rbind(flightsUA,flightsAA)
print(nrow(flightsUAandAA))


Nothing special, just be sure the dataframes have the columns with the same names and types.

A useful tip is to use ``do.call`` in order to merge more than two data frames.
``do.call`` is a function that applies a function to a list of elements.

In [ ]:
message('rBinding 3 data frames and checking the number of rows')
print(nrow(do.call(rbind, list(flightsUA,flightsAA,flightsUAandAA))))

## A useful tip for populating dataframes within a loop
'rbind' is really useful for populating a dataframes, but it can be a bit slow within loops. Each time we append a row to a dataframe within a loop a new copy of a dataframe is stored in memory :( 

The solution is to create a list of lists and then merge them with `do.call rbind` combo. But since ``rbind``, as many native R functions, is slow and not memory-efficient, for large datasets one may want to use
``rbindlist`` function from ``data.table`` package, which does the same operation, but faster. 

Let's compare these approaches using the `system.time` function to see the execution times.

In [ ]:
message('execution time for rbind')
system.time(do.call(rbind, list(flightsUA,flightsAA,flightsUAandAA)))

message('execution time for rbindlist, written in C')
system.time(rbindlist(list(flightsUA,flightsAA,flightsUAandAA)))

And now the example of using rbindlist for populating a data frame.
Let's pretend we forgot how to use `tapply` or `group_by` (`dplyr`) and we want to calculate the average arrival and departure delays per airline.

In [ ]:
Start <- Sys.time()
carriers  <- unique(flights_complete$carrier)
resList <- list()
for (i in 1:length(carriers))
{
    meanDepDelay <- mean(flights_complete[flights_complete$carrier == carriers[i],]$dep_delay)
    meanArrDelay <- mean(flights_complete[flights_complete$carrier == carriers[i],]$arr_delay) 
    resList[[i]] <- list(carriers[i],meanDepDelay,meanArrDelay)
}
DelaysByAirline <- rbindlist(resList)
colnames(DelaysByAirline) <- c("carrier","meanDepDelay","meanArrDelay")
End <- Sys.time()
message('It took us')
print(End-Start)
message('and here is the result for Amercian Airlines')
print(DelaysByAirline[DelaysByAirline$carrier == 'AA',])

message('Same result without messing with loops')
Start <- Sys.time()
flights_complete %>% group_by(carrier)%>%
    summarize(meanDepDelay = mean(dep_delay), meanArrDelay = mean(arr_delay))%>%
        filter(carrier == 'AA') %>% print()
End <- Sys.time()
message('And it took us')
print(End-Start)

In most cases loops are possible to avoid, but if you have to write one, the "list of lists" + `rbindlist` approach may save you a lot of time.

## Merge two tables by a single column // What are the most common destination airports?
The `flights` dataset has destination airports coded, as three-letter airport codes. I'm pretty good at decoding them, but you don't have to be. 

In [ ]:
print(head(airports))

The `airports` table gives us a key! Let's merge the `flights` data with the `airports` data, using `dest` in `flights` and `faa` in `airports`.

In [ ]:
message('This is pretty easy in base R...')
flights_readdest <- merge(flights_complete, airports, by.x='dest', by.y = 'faa', all.x=TRUE)
print(head(flights_readdest))

message('And you can do it in `dplyr` too...')
flights_readdest2 <- left_join(flights_complete, airports, by = c("dest" = "faa"))
print(head(flights_readdest2))

**Why did we use `all.x = TRUE` and `left_join`?**

In [ ]:
print(setdiff(flights$dest, airports$faa))

Well this merged dataset is nice, but do we really need all of this information?

In [ ]:
print(colnames(flights_readdest))

In [ ]:
flights_sm <- select(flights_readdest,origin, dest=name, year, month, day, air_time)
print(head(flights_sm))

Why would you want to ever use `select`? `dplyr` lets you chain operations using the pipes, as discussed above. Let's calculate the average air time for various flight paths, using origin and the readable version of destination airport.

In [ ]:
airtime <- left_join(flights_complete, airports, by = c("dest" = "faa")) %>% 
    select(origin, dest=name, air_time) %>% 
    group_by(origin, dest) %>% 
    summarize(avg_air_time = mean(air_time))

print(head(airtime))
print(dim(airtime))

What's the longest flight from each airport, on average?

In [ ]:
with(airtime, tapply(avg_air_time, origin,  max)) %>% print()
airtime %>% group_by(origin) %>% summarise(max(avg_air_time)) %>% print()

## Pivot Table // Average flight time from origin to destination

Let's put destinations in rows and origins in columns, and have `air_time` as values.

In [ ]:
pvt_airtime <- spread(airtime, origin, avg_air_time)
summary(apply(pvt_airtime, 1, function(x) all(is.na(x))))
print(pvt_airtime)

## Multi-column merge // What's the weather like for departing flights?
Flights...get delayed. What's the first step if you want to know if the departing airport's weather is at all responsible for the delay? Luckily, we have a `weather` dataset for that.

Let's take a look.

In [ ]:
head(weather)
print(intersect(colnames(flights_complete), colnames(weather)))

In [ ]:
flights_weather <- merge(flights_complete, weather, 
                         by=c("year", "month","day","hour", "origin"))
print(dim(flights_complete))
print(dim(flights_weather))


In [ ]:
flights_weather_posdelays <- filter(flights_weather, dep_delay>200)
print(dim(flights_weather_posdelays))

## Arranging a dataframe // What's the weather like for the most and least delayed flights?

Let's sort the `flights_weather` dataframe on `dep_delay` and get data for the top 10 and bottom 10 delays.

In [ ]:
flights_weather %>% arrange(desc(dep_delay)) %>% head(10)

In [ ]:
flights_weather %>% arrange(dep_delay) %>% head(10)

## Some other tidying
## Capitalization issues.

In [ ]:
print(head(tolower(flights_complete$dest)))
print(head(toupper(tolower(flights_complete$dest))))

## Wide to long formatted data

In [ ]:
print(head(flights_complete))

In [ ]:
message('Using `reshape`')
day_delay <- melt(flights_complete, id.vars=c("time_hour"), 
                measure.vars=c("dep_delay","arr_delay"), variable_name = "type_of_delay")
print(head(day_delay))
message('Using `tidyr`')
day_delay <- gather(flights_complete, `dep_delay`,`arr_delay`, 
                    key = "type_of_delay", value="value")
print(head(day_delay))
ggplot(day_delay,aes(x=time_hour,y=value,colour=type_of_delay, group=type_of_delay)) + geom_point()

Well this is a bit hard to read. What about the first entry for each type of delay in each hour? 

## Removing duplicates

In [ ]:
day_delay_first <- distinct(day_delay, time_hour, type_of_delay, .keep_all = TRUE)
print(head(day_delay_first))
ggplot(day_delay_first,aes(x=time_hour,y=value,colour=type_of_delay, group=type_of_delay)) + geom_point()

## An incomplete investigation of NAs 

Let's examine where there are NAs in the `flights` dataset.

In [ ]:
ind <- which(is.na(flights), arr.ind = TRUE)
print(head(ind))
ind2 <- table(ind[,2])
print(ind2)

message('But what do those numbers mean?')
names(ind2) <- colnames(flights)[as.numeric(names(ind2))]
print(ind2)

In [ ]:
flights_incomplete <- flights[!complete.cases(flights),]
print(dim(flights_incomplete))

Do flights with NA departure time also have an NA departure delay?

In [ ]:
print(table(is.na(flights_incomplete$dep_time) & is.na(flights_incomplete$dep_delay)))
print(table(is.na(flights_incomplete$dep_time) | is.na(flights_incomplete$dep_delay)))

Yes.